# Tsyganenko (Geopack and T96)

In [ ]:
import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
from tsyganenko import earth_radius, Trace

## Using the `Trace` class

### Plotting field lines as a schematic

In [ ]:
trace_dt = dt.datetime(2001, 9, 22, 12)
lats = np.arange(10, 90, 10)
lons = 180.
rhos = earth_radius
trace = Trace(lats, lons, rhos, datetime=trace_dt)
ax = trace.plot()
plt.show()
print(trace)

### Making frames for a video of the Dungey Cycle

In [ ]:
trace_dt = dt.datetime(2001,9,22,12)

latlon = np.array([[60,     0],
                   [71,     0],
                   [84,     0],
                   [84,   180],
                   [80,   180],
                   [76,   180],
                   [71.5, 180],
                   [71,   180],
                   [69,   180],
                   [60,   180]])

video_trace = Trace(latlon[:,0], latlon[:,1], earth_radius, datetime = trace_dt)

In [ ]:
ax = video_trace.plot(only_pts=[0,1,6,7,8,9], color='C0')
ax = video_trace.plot(only_pts=[2,3,4,5], color='C1', ls='--')
ax.set(xlim=(12, -22), aspect='equal')
plt.show()
print(video_trace)

In [ ]:
for i, _ in enumerate(latlon):
    fig, ax = plt.subplots(figsize = [16,9])
    video_trace.plot(ax, only_pts=i, lw=5)
    ax.set(xlim=(10,-20), ylim=(-5,10), aspect='equal')
    fig.savefig("Dungey Cycle {:02d}.png".format(i))
    fig.clf()
    plt.close(fig)
    del fig

## Using the `geopack` module

In [ ]:
import geopack_tsyganenko as geopack

You **must** call `recalc_08` before doing anything else

In [ ]:
# Date and time
year = 2000
doy = 1
hour = 0
minute = 0
second = 0

# Solar wind speed
vxgse = -400.
vygse = 0.
vzgse = 0.

geopack.recalc_08(year, doy, hour, minute, second, vxgse, vygse, vzgse)

Then, continue to use the `geopack` routines

In [ ]:
# Execution parameters
lmax = 5000
rlim = 60. 
r0 = 1. 
dsmax = .01
err = .000001

# Direction of the tracing
mapto = 1

# Magnetic activity (for T96, the first four elements of parmod are
# solar wind dynamic pressure (nPa), Dst (nT), BY IMF (nT) and BZ IMF (nT)).
parmod = np.zeros(10, dtype=float)
parmod[0:4] = [2., -8., -2., -5.]

# Start point (rho in Earth radii, colat and lon in degrees)
rho = 1.
lat = 60.
lon = 0.

# Convert lat,lon to geographic cartesian and then gsw
_, _, _, xgeo, ygeo, zgeo = geopack.sphcar_08(
    rho, np.radians(90. - lat), np.radians(lon), 0., 0., 0., 1)
_, _, _, xgsw, ygsw, zgsw = geopack.geogsw_08(
    xgeo, ygeo, zgeo, 0., 0. , 0., 1)

# Trace field line
xfgsw, yfgsw, zfgsw, xarr, yarr, zarr, l_cnt = geopack.trace_08(
    xgsw, ygsw, zgsw, mapto, dsmax, err, rlim, r0, 0, parmod,
    'T96_01', 'IGRF_GSW_08', lmax) 

# Convert back to spherical geographic coords
xfgeo, yfgeo, zfgeo, _, _, _  = geopack.geogsw_08(
    0., 0., 0., xfgsw, yfgsw, zfgsw, -1)
gcR, gdcolat, gdlon, _, _, _ = geopack.sphcar_08(
    0., 0., 0., xfgeo, yfgeo, zfgeo, -1)

print('** START: {:6.3f}, {:6.3f}, {:6.3f}'.format(
    lat, lon, 1.))
print('** STOP:  {:6.3f}, {:6.3f}, {:6.3f}'.format(
    90.-np.degrees(gdcolat), np.degrees(gdlon), gcR))

A quick plot to check the results:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot coordinate system
ax.plot3D([0, 1], [0, 0], [0, 0], 'b')
ax.plot3D([0, 0], [0, 1], [0, 0], 'g')
ax.plot3D([0, 0], [0, 0], [0, 1], 'r')

# First plot a nice sphere for the Earth
u = np.linspace(0, 2 * np.pi, 179)
v = np.linspace(0, np.pi, 179)
tx = np.outer(np.cos(u), np.sin(v))
ty = np.outer(np.sin(u), np.sin(v))
tz = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_surface(tx, ty, tz, rstride=10, cstride=10, color='grey',
                alpha=.5, zorder=2, linewidth=0.5)

# Then plot the traced field line
latarr = np.arange(10, 90, 10, dtype=float)
lonarr = np.array([0., 180.], dtype=float)
rho = 1.

for lon in lonarr:
    for lat in latarr:
        _, _, _, xgeo, ygeo, zgeo = geopack.sphcar_08(
            rho, np.radians(90.-lat), np.radians(lon), 0., 0., 0., 1)
        _, _, _, xgsw, ygsw, zgsw  = geopack.geogsw_08(
            xgeo, ygeo, zgeo, 0., 0., 0., 1)
        
        xfgsw, yfgsw, zfgsw, xarr, yarr, zarr, l_cnt = geopack.trace_08(
            xgsw, ygsw, zgsw, mapto, dsmax, err, rlim, r0, 0, parmod,
            'T96_01', 'IGRF_GSW_08', lmax)

        # Iterate through the array, converting to geographic coordinates.
        for i in np.arange(l_cnt):
            xgeo, ygeo, zgeo, _, _, _ = geopack.geogsw_08(
                0., 0., 0., xarr[i], yarr[i], zarr[i], -1)
            xarr[i], yarr[i], zarr[i] = xgeo, ygeo, zgeo
            
        ax.plot3D(xarr[0:l_cnt], yarr[0:l_cnt], zarr[0:l_cnt],
                  zorder=3, linewidth=2, color='C1')

# Set plot limits
xyzlim = 4
_ = ax.set(xlim3d=[-xyzlim, xyzlim],
           ylim3d=[-xyzlim, xyzlim],
           zlim3d=[-xyzlim, xyzlim])